In [3]:
!vllm serve "Qwen/Qwen2.5-7B-Instruct"

INFO 12-11 21:51:57 api_server.py:585] vLLM API server version 0.6.4.post1
INFO 12-11 21:51:57 api_server.py:586] args: Namespace(subparser='serve', model_tag='Qwen/Qwen2.5-7B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='Qwen/Qwen2.5-7B-Instruct', task='auto', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', chat_template_text_format='string', trust_remote_code=False, allowed_local_media_path=None, download_dir=None, load_format='auto',

## Dependencies

In [1]:
import pandas as pd
from transformers import AutoTokenizer
from utils import prepare_items, get_detailed_instruct, get_new_queries, save_data

/home/hc4293/miniconda3/envs/nlpenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## EDA

In [ ]:
# Configuration
IS_SUBMISSION = True
model_path = "Qwen/Qwen2.5-7B-Instruct"
LORA_PATH = "/kaggle/input/lora-14b-1126/transformers/default/1"
query_max_len, doc_max_len = 320, 48
examples_prefix = ""
task = "Given a math multiple-choice problem with a student's wrong answer, retrieve the mathematical misconceptions"

# Load data
df_train = pd.read_csv("./data/train.csv").fillna(-1).sample(10, random_state=42).reset_index(drop=True)
df_test = pd.read_csv("./data/test.csv")
df_misconception_mapping = pd.read_csv("./data/misconception_mapping.csv")

df_ret = df_test if IS_SUBMISSION else df_train
df_input, target_ids = prepare_items(df_ret, IS_SUBMISSION)


### Train data

In [6]:
df_train.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,1700,321,Divide two decimals with the same number of de...,224,Multiplying and Dividing with Decimals,D,\( 0.9 \div 0.3= \),\( 0.3 \),\( 9.3 \),\( 0.333 \ldots \),\( 3 \),153.0,-1.0,2359.0,-1.0
1,1488,469,Understand the notation for powers,245,"Squares, Cubes, etc",C,To calculate \( 53^{2} \) you need to do...,\( 53+2 \),\( 53 \times 2 \),\( 53 \times 53 \),\( 532 \times 1 \),2445.0,2316.0,-1.0,-1.0
2,921,219,Round numbers to three or more decimal places,214,Rounding to Decimal Places,A,What is \( 20.15349 \) rounded to \( 3 \) deci...,\( 20.153 \),\( 20.15 \),\( 20.154 \),\( 20.253 \),-1.0,2392.0,1988.0,2330.0
3,275,306,Subtract decimals where the numbers involved h...,223,Adding and Subtracting with Decimals,A,\( 50.09-0.1= \),\( 49.99 \),\( 50.99 \),\( 50.08 \),\( 38.98 \),-1.0,699.0,2346.0,-1.0
4,416,703,Express pictorial representations of objects a...,334,Writing Ratios,C,Tom says for every one circle there are two sq...,Only\nTom,Only Katie,Both Tom and Katie,Neither is correct,-1.0,1846.0,-1.0,1150.0


## Input for the model

In [10]:
# Prepare queries
queries = [get_detailed_instruct(task, q) for q in df_input['Prompt']]
documents = df_misconception_mapping['MisconceptionName'].tolist()

# Tokenize queries
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)
new_query_max_len, new_queries = get_new_queries(queries, query_max_len, examples_prefix, tokenizer)

# Save data
save_data(new_queries, documents)

OSError: Incorrect path_or_model_id: '/kaggle/input/lora-14b-1126/transformers/default/1'. Please provide either the path to a local folder or the repo_id of a model on the Hub.